Preprocessed.zip file are located in Google drive, in order to train a model preprocessed file should be created under the local disk of the colab

In [1]:
!cp "/content/drive/MyDrive/wifi_csi/wifi_csi/preprocessed.zip" /content/

In [2]:
!unzip -q /content/preprocessed.zip -d /content

Importing necessary libraries. Select a GPU for high performance

CNN+LSTM Hybrid Model With Additional Signal Processing

In [113]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# =============================================================================
# 1. SETUP & CONFIGURATION
# =============================================================================
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

CONFIG = {
    # Paths
    'dataset_path': '/content/drive/MyDrive/wifi_csi',
    'preprocessed_dir': '/content/preprocessed',
    'annotation_file': '/content/annotation.csv',

    # Training
    'batch_size': 32,
    'lr': 5e-4,
    'epochs': 30,
    'num_classes': 6,
    'window': 1500,  # Time window to use
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',

    # Augmentation
    'use_augmentation': True,
    'mixup_alpha': 0.2,
    'time_mask_ratio': 0.1,  # Mask 10% of time steps
    'freq_mask_ratio': 0.1,  # Mask 10% of frequency bins

    # Domain Adaptation
    'use_coral': True,
    'coral_weight': 0.5,

    # Regularization
    'label_smoothing': 0.1,
    'dropout': 0.4,
    'weight_decay': 1e-4,
}

print(f"🚀 Running on Device: {CONFIG['device']}")

# =============================================================================
# 2. DATASET CLASS (FIXED FOR YOUR DATA FORMAT)
# =============================================================================

class WimansDatasetFixed(Dataset):
    """
    Dataset class compatible with preprocessed data format: (2, 3000, 270)
    - Channel 0: Amplitude
    - Channel 1: Phase
    - 270 = 9 antenna pairs × 30 subcarriers (flattened)
    """
    def __init__(self, df, root_dir, config, is_train=False):
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.config = config
        self.is_train = is_train
        self.window = config['window']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file_path = os.path.join(self.root_dir, f"{row['label']}.npy")
        label = row['number_of_users']

        try:
            # Load preprocessed data: (2, 3000, 270)
            data = np.load(file_path).astype(np.float32)

            # Validate shape
            if data.shape[0] != 2:
                raise ValueError(f"Expected 2 channels, got {data.shape[0]}")

            total_time = data.shape[1]  # Should be 3000
            n_features = data.shape[2]  # Should be 270

            # Time slicing
            if self.is_train:
                # Random slice during training
                if total_time > self.window:
                    start = np.random.randint(0, total_time - self.window)
                else:
                    start = 0
            else:
                # Center slice during validation/test
                start = max(0, (total_time - self.window) // 2)

            # Handle sequences shorter than window
            if total_time < self.window:
                pad_amt = self.window - total_time
                data = np.pad(data, ((0, 0), (0, pad_amt), (0, 0)), mode='edge')
                start = 0

            # Extract window: (2, window, 270)
            data = data[:, start:start+self.window, :]

            # Reshape 270 back to (9, 30) for spatial structure
            # New shape: (2, window, 9, 30)
            amp = data[0].reshape(self.window, 9, 30)   # (window, 9, 30)
            phase = data[1].reshape(self.window, 9, 30) # (window, 9, 30)

            # Transpose to channel-first: (9, window, 30) for each
            amp = amp.transpose(1, 0, 2)     # (9, window, 30)
            phase = phase.transpose(1, 0, 2) # (9, window, 30)

            # Apply augmentations during training
            if self.is_train and self.config['use_augmentation']:
                amp, phase = self._apply_augmentations(amp, phase)

            # Stack amplitude and phase: (18, window, 30)
            processed_data = np.concatenate([amp, phase], axis=0)

            # Per-sample normalization (separate for amp and phase)
            processed_data = self._normalize(processed_data)

            return torch.from_numpy(processed_data), torch.tensor(label, dtype=torch.long)

        except Exception as e:
            # Return zeros on error (will be filtered by model)
            print(f"Error loading {file_path}: {e}")
            return torch.zeros(18, self.window, 30), torch.tensor(label, dtype=torch.long)

    def _apply_augmentations(self, amp, phase):
        """Apply data augmentations."""
        # 1. Time masking (SpecAugment style)
        if np.random.random() < 0.5:
            mask_len = int(self.window * self.config['time_mask_ratio'])
            mask_start = np.random.randint(0, self.window - mask_len)
            amp[:, mask_start:mask_start+mask_len, :] = 0
            phase[:, mask_start:mask_start+mask_len, :] = 0

        # 2. Frequency masking
        if np.random.random() < 0.5:
            mask_len = int(30 * self.config['freq_mask_ratio'])
            mask_start = np.random.randint(0, 30 - mask_len)
            amp[:, :, mask_start:mask_start+mask_len] = 0
            phase[:, :, mask_start:mask_start+mask_len] = 0

        # 3. Random amplitude scaling (simulates AGC variation)
        if np.random.random() < 0.5:
            scale = np.random.uniform(0.8, 1.2)
            amp = amp * scale

        # 4. Random time reversal
        if np.random.random() < 0.5:
            amp = amp[:, ::-1, :].copy()
            phase = phase[:, ::-1, :].copy()

        # 5. Add small Gaussian noise
        if np.random.random() < 0.5:
            noise_amp = np.random.randn(*amp.shape) * 0.01
            noise_phase = np.random.randn(*phase.shape) * 0.01
            amp = amp + noise_amp
            phase = phase + noise_phase

        return amp, phase

    def _normalize(self, data):
        """Normalize amplitude and phase separately."""
        # Amplitude: channels 0-8
        amp = data[:9]
        amp_mean = amp.mean()
        amp_std = amp.std() + 1e-8
        data[:9] = (amp - amp_mean) / amp_std

        # Phase: channels 9-17
        phase = data[9:]
        phase_mean = phase.mean()
        phase_std = phase.std() + 1e-8
        data[9:] = (phase - phase_mean) / phase_std

        return data.astype(np.float32)


# =============================================================================
# 3. IMPROVED ARCHITECTURE WITH ATTENTION
# =============================================================================

class ChannelAttention(nn.Module):
    """Squeeze-and-Excitation style channel attention."""
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        # Average pooling path
        avg_out = self.fc(self.avg_pool(x).view(b, c))
        # Max pooling path
        max_out = self.fc(self.max_pool(x).view(b, c))
        # Combine
        out = self.sigmoid(avg_out + max_out).view(b, c, 1, 1)
        return x * out.expand_as(x)

class ImprovedCNNLSTM(nn.Module):
    """
    Enhanced CNN+LSTM architecture.
    Input: (batch, 18, time, 30) - 9 amp + 9 phase channels
    """
    def __init__(self, num_classes=6, input_channels=18, dropout=0.4):
        super().__init__()

        # CNN Feature Extractor
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=(7, 3), stride=(2, 1), padding=(3, 1)),
            nn.BatchNorm2d(32),
            nn.GELU(),
            nn.MaxPool2d((2, 1)),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 1), padding=(2, 1)),
            nn.BatchNorm2d(64),
            nn.GELU(),
            nn.MaxPool2d((2, 1)),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.GELU(),
        )

        # Channel Attention after conv layers
        self.channel_attn = ChannelAttention(128)

        # Adaptive pooling to collapse frequency dimension
        self.adaptive_pool = nn.AdaptiveAvgPool2d((None, 1))

        # Layer normalization before LSTM
        self.pre_lstm_norm = nn.LayerNorm(128)

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=128,
            num_layers=2,
            batch_first=True,
            dropout=dropout if dropout > 0 else 0,
            bidirectional=True
        )

        # Classification head
        self.classifier = nn.Sequential(
            nn.LayerNorm(256),  # 256 = 128 * 2 (bidirectional)
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.Dropout(dropout / 2),
            nn.Linear(128, num_classes)
        )

        # Feature output for CORAL loss
        self.feature_dim = 256

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x, return_features=False):
        # CNN forward pass
        x = self.conv1(x)   # (B, 32, T/4, 30)
        x = self.conv2(x)   # (B, 64, T/16, 30)
        x = self.conv3(x)   # (B, 128, T/16, 30)

        # Channel attention
        x = self.channel_attn(x)

        # Pool frequency dimension
        x = self.adaptive_pool(x)  # (B, 128, T/16, 1)

        # Reshape for LSTM: (B, T, C)
        x = x.squeeze(3).permute(0, 2, 1)

        # Layer norm
        x = self.pre_lstm_norm(x)

        # LSTM
        self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)

        # Get features from last time step
        features = lstm_out[:, -1, :]  # (B, 256)

        if return_features:
            return features

        # Classification
        logits = self.classifier(features)
        return logits

# =============================================================================
# 4. LOSS FUNCTIONS
# =============================================================================

class LabelSmoothingCrossEntropy(nn.Module):
    """Cross entropy with label smoothing."""
    def __init__(self, smoothing=0.1, weight=None):
        super().__init__()
        self.smoothing = smoothing
        self.weight = weight

    def forward(self, pred, target):
        n_classes = pred.size(-1)

        with torch.no_grad():
            smooth_labels = torch.zeros_like(pred)
            smooth_labels.fill_(self.smoothing / (n_classes - 1))
            smooth_labels.scatter_(1, target.unsqueeze(1), 1 - self.smoothing)

        log_probs = F.log_softmax(pred, dim=-1)
        loss = -(smooth_labels * log_probs).sum(dim=-1)

        if self.weight is not None:
            weight = self.weight[target]
            loss = loss * weight

        return loss.mean()

class CORALLoss(nn.Module):
    """Deep CORAL loss for domain adaptation."""
    def __init__(self):
        super().__init__()

    def forward(self, source, target):
        d = source.size(1)
        ns, nt = source.size(0), target.size(0)

        # Source covariance
        source_centered = source - source.mean(0, keepdim=True)
        cs = (source_centered.T @ source_centered) / (ns - 1 + 1e-8)

        # Target covariance
        target_centered = target - target.mean(0, keepdim=True)
        ct = (target_centered.T @ target_centered) / (nt - 1 + 1e-8)

        # CORAL loss
        loss = torch.sum((cs - ct) ** 2) / (4 * d * d)
        return loss

# =============================================================================
# 5. MIXUP AUGMENTATION
# =============================================================================

def mixup_data(x, y, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]

    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# =============================================================================
# 6. TRAINING FUNCTIONS
# =============================================================================

def train_epoch(model, loader, criterion, optimizer, config,
                target_loader=None, coral_loss_fn=None):
    model.train()
    total_loss, total_cls_loss, total_coral_loss = 0, 0, 0
    correct, total = 0, 0

    target_iter = iter(target_loader) if target_loader else None

    pbar = tqdm(loader, desc="Training", leave=False)
    for x, y in pbar:
        x, y = x.to(config['device']), y.to(config['device'])

        # Skip batches with all zeros (error samples)
        if x.abs().sum() == 0:
            continue

        # Mixup augmentation
        use_mixup = config['use_augmentation'] and config['mixup_alpha'] > 0
        if use_mixup and np.random.random() < 0.5:
            x, y_a, y_b, lam = mixup_data(x, y, config['mixup_alpha'])
        else:
            use_mixup = False

        optimizer.zero_grad()

        # Forward pass with CORAL
        if config['use_coral'] and target_loader:
            outputs = model(x)
            source_features = model(x, return_features=True)

            try:
                x_target, _ = next(target_iter)
            except StopIteration:
                target_iter = iter(target_loader)
                x_target, _ = next(target_iter)

            x_target = x_target.to(config['device'])
            if x_target.abs().sum() > 0:  # Skip if all zeros
                target_features = model(x_target, return_features=True)
                coral = coral_loss_fn(source_features, target_features)
            else:
                coral = torch.tensor(0.0).to(config['device'])
        else:
            outputs = model(x)
            coral = torch.tensor(0.0).to(config['device'])

        # Classification loss
        if use_mixup:
            cls_loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
        else:
            cls_loss = criterion(outputs, y)

        # Total loss
        loss = cls_loss + config['coral_weight'] * coral

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        total_cls_loss += cls_loss.item()
        total_coral_loss += coral.item()

        _, predicted = outputs.max(1)
        total += y.size(0)
        if use_mixup:
            correct += (lam * predicted.eq(y_a).float() +
                       (1-lam) * predicted.eq(y_b).float()).sum().item()
        else:
            correct += predicted.eq(y).sum().item()

        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.1f}%'
        })

    n_batches = len(loader)
    return (total_loss / n_batches, total_cls_loss / n_batches,
            total_coral_loss / n_batches, 100. * correct / total)


def validate(model, loader, criterion, config):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(config['device']), y.to(config['device'])

            # Skip error samples
            if x.abs().sum() == 0:
                continue

            outputs = model(x)
            loss = criterion(outputs, y)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = 100. * correct / total if total > 0 else 0
    avg_loss = total_loss / len(loader) if len(loader) > 0 else 0
    return avg_loss, acc, all_preds, all_labels


# =============================================================================
# 7. MAIN FUNCTION
# =============================================================================

def main():
    print("=" * 60)
    print("FIXED CNN+LSTM PIPELINE FOR WIFI CSI HUMAN COUNTING")
    print("Data format: (2, 3000, 270) - preprocessed amplitude & phase")
    print("=" * 60)

    # Load annotations
    df = pd.read_csv(CONFIG['annotation_file'])
    print(f"Total samples: {len(df)}")
    print(f"Available Environments: {df['environment'].unique()}")

    # Domain split: train on classroom + meeting_room, test on empty_room
    train_envs = ['classroom', 'meeting_room']
    test_env = 'empty_room'

    train_val_df = df[df['environment'].isin(train_envs)].reset_index(drop=True)
    test_df = df[df['environment'] == test_env].reset_index(drop=True)

    # Stratified validation split
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    for train_idx, val_idx in split.split(train_val_df, train_val_df['number_of_users']):
        train_df = train_val_df.iloc[train_idx]
        val_df = train_val_df.iloc[val_idx]

    print(f"\nDOMAIN SPLIT:")
    print(f"  TRAIN: {train_envs} | {len(train_df)} samples")
    print(f"  VAL:   {train_envs} | {len(val_df)} samples")
    print(f"  TEST:  {test_env} | {len(test_df)} samples (UNSEEN)")

    # Create datasets
    train_ds = WimansDatasetFixed(train_df, CONFIG['preprocessed_dir'], CONFIG, is_train=True)
    val_ds = WimansDatasetFixed(val_df, CONFIG['preprocessed_dir'], CONFIG, is_train=False)
    test_ds = WimansDatasetFixed(test_df, CONFIG['preprocessed_dir'], CONFIG, is_train=False)
    target_ds = WimansDatasetFixed(test_df, CONFIG['preprocessed_dir'], CONFIG, is_train=False)

    # Create dataloaders
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'],
                             shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'],
                           shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=CONFIG['batch_size'],
                            shuffle=False, num_workers=2, pin_memory=True)
    target_loader = DataLoader(target_ds, batch_size=CONFIG['batch_size'],
                              shuffle=True, num_workers=2, pin_memory=True)

    # Initialize model
    model = ImprovedCNNLSTM(
        num_classes=CONFIG['num_classes'],
        input_channels=18,
        dropout=CONFIG['dropout']
    ).to(CONFIG['device'])

    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n📊 Model: {total_params:,} parameters")

    # Class weights
    class_counts = train_df['number_of_users'].value_counts().sort_index().values
    weights = torch.FloatTensor(1.0 / class_counts).to(CONFIG['device'])
    weights = weights / weights.sum()
    print(f"Class weights: {weights.cpu().numpy().round(4)}")

    # Loss and optimizer
    criterion = LabelSmoothingCrossEntropy(
        smoothing=CONFIG['label_smoothing'],
        weight=weights
    )
    coral_loss_fn = CORALLoss() if CONFIG['use_coral'] else None

    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'],
                           weight_decay=CONFIG['weight_decay'])
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-6
    )

    # Training loop
    print("\n🔥 Starting Training...")
    print("-" * 60)

    best_val_acc = 0
    best_test_acc = 0

    for epoch in range(CONFIG['epochs']):
        train_loss, cls_loss, coral_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, CONFIG,
            target_loader if CONFIG['use_coral'] else None,
            coral_loss_fn
        )

        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, CONFIG)
        test_loss, test_acc, _, _ = validate(model, test_loader, criterion, CONFIG)

        scheduler.step()

        print(f"Epoch {epoch+1:02d}/{CONFIG['epochs']} | "
              f"Train: {train_acc:.1f}% | Val: {val_acc:.1f}% | "
              f"Test: {test_acc:.1f}% | LR: {scheduler.get_last_lr()[0]:.2e}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        if test_acc > best_test_acc:
            best_test_acc = test_acc

    # Final evaluation
    print("\n" + "=" * 60)
    print("🏆 RESULTS")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"Best Test Accuracy (Unseen): {best_test_acc:.2f}%")
    print("=" * 60)

    # Load best model and evaluate
    model.load_state_dict(torch.load('best_model.pth'))
    _, final_acc, preds, labels = validate(model, test_loader, criterion, CONFIG)

    print(f"\n📊 Final Test Results ({test_env}):")
    print(classification_report(labels, preds, digits=3, zero_division=0))

    return model


if __name__ == "__main__":
    model = main()

🚀 Running on Device: cuda
FIXED CNN+LSTM PIPELINE FOR WIFI CSI HUMAN COUNTING
Data format: (2, 3000, 270) - preprocessed amplitude & phase
Total samples: 11286
Available Environments: ['classroom' 'meeting_room' 'empty_room']

DOMAIN SPLIT:
  TRAIN: ['classroom', 'meeting_room'] | 6395 samples
  VAL:   ['classroom', 'meeting_room'] | 1129 samples
  TEST:  empty_room | 3762 samples (UNSEEN)

📊 Model: 819,302 parameters
Class weights: [0.4005 0.0666 0.1332 0.1332 0.1332 0.1332]

🔥 Starting Training...
------------------------------------------------------------


Epoch 01/30 | Train: 33.7% | Val: 54.6% | Test: 24.0% | LR: 4.88e-04


Epoch 02/30 | Train: 52.8% | Val: 65.5% | Test: 30.4% | LR: 4.52e-04


Epoch 03/30 | Train: 60.7% | Val: 71.8% | Test: 32.7% | LR: 3.97e-04


Epoch 04/30 | Train: 64.3% | Val: 72.9% | Test: 27.7% | LR: 3.28e-04


...

Epoch 26/30 | Train: 83.6% | Val: 89.9% | Test: 39.8% | LR: 4.87e-05


Epoch 27/30 | Train: 84.7% | Val: 89.7% | Test: 39.8% | LR: 2.82e-05


Epoch 28/30 | Train: 86.0% | Val: 90.2% | Test: 42.1% | LR: 1.32e-05


Epoch 29/30 | Train: 87.1% | Val: 90.3% | Test: 41.5% | LR: 4.07e-06


Epoch 30/30 | Train: 86.8% | Val: 90.2% | Test: 41.6% | LR: 5.00e-04

🏆 RESULTS
Best Validation Accuracy: 90.35%
Best Test Accuracy (Unseen): 43.57%

📊 Final Test Results (empty_room):
              precision    recall  f1-score   support

           0      0.818     0.500     0.621       198
           1      0.485     0.466     0.475      1188
           2      0.507     0.527     0.517       594
           3      0.188     0.093     0.124       594
           4      0.367     0.300     0.330       594
           5      0.327     0.608     0.425       594

    accuracy                          0.415      3762
   macro avg      0.449     0.416     0.415      3762
weighted avg      0.415     0.415     0.403      3762

